In [ ]:
!pip install -q transformers datasets librosa

In [ ]:
from datasets import load_dataset


# Загрузка dataset CEDR.
Корпус для выявления эмоций в предложениях русскоязычных текстов из разных социальных источников (CEDR) содержит 9410 комментариев, помеченных по 5 категориям эмоций (радость, грусть, удивление, страх и гнев).

In [ ]:
cedr= load_dataset('cedr')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/757k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/188k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7528 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1882 [00:00<?, ? examples/s]

In [ ]:
cedr

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'source'],
        num_rows: 7528
    })
    test: Dataset({
        features: ['text', 'labels', 'source'],
        num_rows: 1882
    })
})

In [ ]:
# загрузим набор данных train
df = cedr['train'].to_pandas()
df_cedr= df[['text','labels']]
df_cedr

,text,labels
0,Суровый гималайский медведь .,[]
1,"Так, обнаружено несколько проблем с дисплеем (...",[]
2,У меня остается только один вопрос - является ...,[2]
3,Забавно как люди в возрасте удивляются входящи...,[0]
4,"Издание The Register отмечает, что у владельце...",[]
...,...,...
7523,Ухаха какая я злая!,[4]
7524,Я очень рада вас видеть.,[0]
7525,Жизнь боль.,[1]
7526,- Не удивительно .,[]


# Значение меток
0: "joy", 1: "sadness", 2: "surprise", 3: "fear", 4: "anger" , пустой список - "neutral"





In [ ]:
labels = ["neutral", "joy", "sadness", "surprise","fear","anger"]


In [ ]:
def colum( df,labels):
  for col in labels:
    if df.
    df[col]=

In [ ]:
codes = {0: "joy", 1: "sadness", 2: "surprise", 3: "fear", 4: "anger"}
#  переведем метки в бинарное предстваление
for emotion_code, emotion_name in codes.items():
    df_cedr[emotion_name] = df_cedr['labels'].apply(lambda x: int(emotion_code in x))

df_cedr

,text,labels,joy,sadness,surprise,fear,anger
0,Суровый гималайский медведь .,[],0,0,0,0,0
1,"Так, обнаружено несколько проблем с дисплеем (...",[],0,0,0,0,0
2,У меня остается только один вопрос - является ...,[2],0,0,1,0,0
3,Забавно как люди в возрасте удивляются входящи...,[0],1,0,0,0,0
4,"Издание The Register отмечает, что у владельце...",[],0,0,0,0,0
...,...,...,...,...,...,...,...
7523,Ухаха какая я злая!,[4],0,0,0,0,1
7524,Я очень рада вас видеть.,[0],1,0,0,0,0
7525,Жизнь боль.,[1],0,1,0,0,0
7526,- Не удивительно .,[],0,0,0,0,0


In [ ]:
# добавим нейтральный класс
df_cedr['neutral']= df_cedr['joy']+df_cedr['anger']+df_cedr['fear']+df_cedr['sadness']+df_cedr['surprise']
df_cedr['neutral'] = df_cedr['neutral'].map(lambda x: 0 if x > 0 else 1)
df_cedr

,text,labels,joy,sadness,surprise,fear,anger,neutral,disgust
0,Суровый гималайский медведь .,[],0,0,0,0,0,1,0
1,"Так, обнаружено несколько проблем с дисплеем (...",[],0,0,0,0,0,1,0
2,У меня остается только один вопрос - является ...,[2],0,0,1,0,0,0,0
3,Забавно как люди в возрасте удивляются входящи...,[0],1,0,0,0,0,0,0
4,"Издание The Register отмечает, что у владельце...",[],0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
7523,Ухаха какая я злая!,[4],0,0,0,0,1,0,0
7524,Я очень рада вас видеть.,[0],1,0,0,0,0,0,0
7525,Жизнь боль.,[1],0,1,0,0,0,0,0
7526,- Не удивительно .,[],0,0,0,0,0,1,0


In [ ]:
# Добавим класс disgust т.к его нет в наборе заполним нулями.
df_cedr = df_cedr.assign(disgust=0)
df_cedr

,text,labels,joy,sadness,surprise,fear,anger,neutral,disgust
0,Суровый гималайский медведь .,[],0,0,0,0,0,1,0
1,"Так, обнаружено несколько проблем с дисплеем (...",[],0,0,0,0,0,1,0
2,У меня остается только один вопрос - является ...,[2],0,0,1,0,0,0,0
3,Забавно как люди в возрасте удивляются входящи...,[0],1,0,0,0,0,0,0
4,"Издание The Register отмечает, что у владельце...",[],0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
7523,Ухаха какая я злая!,[4],0,0,0,0,1,0,0
7524,Я очень рада вас видеть.,[0],1,0,0,0,0,0,0
7525,Жизнь боль.,[1],0,1,0,0,0,0,0
7526,- Не удивительно .,[],0,0,0,0,0,1,0


In [ ]:
# удалим ненижную колонку
df_cedr_1 = df_cedr[['text',"anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]]
df_cedr_1

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,Суровый гималайский медведь .,0,0,0,0,0,0,1
1,"Так, обнаружено несколько проблем с дисплеем (...",0,0,0,0,0,0,1
2,У меня остается только один вопрос - является ...,0,0,0,0,0,1,0
3,Забавно как люди в возрасте удивляются входящи...,0,0,0,1,0,0,0
4,"Издание The Register отмечает, что у владельце...",0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
7523,Ухаха какая я злая!,1,0,0,0,0,0,0
7524,Я очень рада вас видеть.,0,0,0,1,0,0,0
7525,Жизнь боль.,0,0,0,0,1,0,0
7526,- Не удивительно .,0,0,0,0,0,0,1


In [ ]:
import pandas as pd
 # загружаем датасет полученный путем добавления перефразированных данных
df_em = pd.read_csv('train_and_peref.csv')
df_em

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,"Моя любимая еда — это все, что мне не приходил...",0,0,0,0,0,0,1
1,"Теперь, если он покончит с собой, все будут ду...",0,0,0,0,0,0,1
2,КАКОГО НАХРАНА БЭЙЛЕССКАЯ ИЗОИНЦИЯ?,1,0,0,0,0,0,0
3,Чтобы она почувствовала угрозу,0,0,1,0,0,0,0
4,Грязные южные дрочники,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
60652,"Я думала, что я наркоман.",0,0,0,0,0,1,0
60653,"Не знаю, меня устраивает религия, которая одоб...",0,0,0,0,0,1,0
60654,"""Как выглядели """"ИМЯ"""" и """"ИМЯ"""" вчера вечером?""",0,0,0,0,0,1,0
60655,Твоя паранойя в этом смысле мне кажется шизофр...,0,0,0,0,0,1,1


In [ ]:
# Сформируем общий датасет
df_con= pd.concat([df_em, df_cedr_1], ignore_index=True)
df_con

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,"Моя любимая еда — это все, что мне не приходил...",0,0,0,0,0,0,1
1,"Теперь, если он покончит с собой, все будут ду...",0,0,0,0,0,0,1
2,КАКОГО НАХРАНА БЭЙЛЕССКАЯ ИЗОИНЦИЯ?,1,0,0,0,0,0,0
3,Чтобы она почувствовала угрозу,0,0,1,0,0,0,0
4,Грязные южные дрочники,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
68180,Ухаха какая я злая!,1,0,0,0,0,0,0
68181,Я очень рада вас видеть.,0,0,0,1,0,0,0
68182,Жизнь боль.,0,0,0,0,1,0,0
68183,- Не удивительно .,0,0,0,0,0,0,1


In [ ]:
df_con.to_csv('df_cedr_para.csv', index=False)